In [1]:
#  测试通过案例
import pandas as pd
train = pd.read_feather('/data/home/lichengzhang/zhoujun/HaimianData/20250325_split/train74_5_300750.SZ/20200427.ftr')
test = pd.read_feather('/data/home/lichengzhang/zhoujun/HaimianData/20250325_split/test74_5_300750.SZ/20200427.ftr')
# Data Pre
cat_cols = []
num_cols = [f'factor_{i}' for i in range(1,113)]
train['y'] = train['y60_duo'].apply(lambda x: 1 if x > 0.0022 else 0)
test['y'] = test['y60_duo'].apply(lambda x: 1 if x > 0.0022 else 0)

In [ ]:
from config import DataConfig, TrainConfig
from data import StockDataModule
from omegaconf import OmegaConf 
from models.mlp import MLPModel, MLPConfig
from utils.loss import MultiTaskLoss
from haimian_model import HaimianModel

config = DataConfig(
        continuous_cols=["factor_0", "factor_5", "factor_6"],
        target_cols=["y", "y60_duo"],
        task_types={"y": "classification", "y60_duo": "regression"},
        categorical_cols=None,
        category_col="factor_0",
        target_category=7,
        window_len=1,
        padding_value=0.0,
        split_ratio=0.1,
        split_type="time",
        split_start=0.9
    )

models_config = MLPConfig(
    layers = "32-32"
)
config = OmegaConf.structured(config)
models_config = OmegaConf.structured(models_config)

# 合并所有参数
config = OmegaConf.merge(
    OmegaConf.to_container(config), 
    OmegaConf.to_container(models_config)
    )

# # 数据模块
# data_module = StockDataModule(
#     train = train, 
#     config = config, 
#     validation = None, # 空的话从训练集中切分
#     train_sampler = True, #使用
#     verbose = True,
# )
# infer_config = data_module.infer_config(config)
# infer_config = OmegaConf.structured(infer_config)
# config = OmegaConf.merge(config, OmegaConf.to_container(infer_config))
# data_module.setup() 
# traindataset = data_module.train_dataset
# print(traindataset[1])




ConfigAttributeError: Missing key batch_size
    full_key: batch_size
    object_type=dict

In [ ]:
data_config = DataConfig(
        categorical_cols=["factor_0"],
        continuous_cols=[ "factor_5", "factor_6"],
        target_cols=["y", "y60_duo"],
        task_types={"y": "classification", "y60_duo": "regression"},
        category_col="factor_0",
        target_category=7,
        window_len=1,
        padding_value=0.0,
        split_ratio=0.1,
        split_type="time",
        split_start=0.9
    )

model_config = MLPConfig(
    layers = "32-32"
)

trainer_config = TrainConfig(
    batch_size=256
)

haimian_model = HaimianModel(
        data_config=data_config,
        model_config=model_config,
        trainer_config=trainer_config,
        verbose=True
    )
task_weights = {"y": 1.0, "y60_duo": 1.0}
#自定义的损失函数
loss_fn = MultiTaskLoss(task_types=data_config.task_types, task_weights=task_weights)
haimian_model.fit(train=train)

TypeError: DataConfig.__init__() got an unexpected keyword argument 'batch_size'